<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 2<br/>*POS taggers* pour le français dans spaCy et NLTK

**Objectif**

Comparer l'étiqueteur morphosyntaxique français prêt-à-l'emploi de spaCy avec deux étiqueteurs entraînés, l'un dans spaCy et l'autre dans NLTK.

## 1. Installation et test de spaCy

La boîte à outils spaCy est une librairie Python *open source* pour le TAL, dédiée à un usage en production. Les documents suivants vous seront utiles :
* comment [installer](https://spacy.io/usage) spaCy
* comment [télécharger un modèle](https://spacy.io/usage/models) pour une langue donnée (on appelle ces modèles des *trained pipelines* car ils enchaînent plusieurs traitements)
* comment faire les [premiers pas](https://spacy.io/usage/spacy-101) dans l'utilisation de spaCy

Veuillez installer spaCy, puis la *pipeline* pour le français appelée `fr_core_news_sm`.  Si vous utilisez *conda*, installez spaCy dans l'environnement du cours TAL.

In [1]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline
import tqdm # permet l'affichage d'une barre de progression

**1a.** Une pipeline effectue un ensemble de traitements d'un texte en lui ajoutant des annotations.  Les traitements effectués par la pipeline `fr_core_news_sm` sont [documentés ici](https://spacy.io/models/fr#fr_core_news_sm).  La liste des traitements d'une pipeline figure dans son attribut `.pipe_names`.  On peut activer ou désactiver un traitement T avec, respectivement, les méthodes `.disable_pipe(T)` et `.enable_pipe(T)` appliquées à la pipeline.

* Veuillez afficher les traitements disponibles dans la pipeline `fr_core_news_sm` chargée ci-dessus sous le nom de `nlp` .
* Veuillez désactiver tous les traitements sauf `tok2vec` et `morphologizer` (on fait cela pour accélerer le traitement).
* Vérifiez que la désactivation a bien fonctionné en affichant les traitements activés.

In [2]:
# Veuillez écrire votre code ici.
print("Traitements disponibles : " + ",".join(nlp.pipe_names))
nlp.disable_pipes("parser", "attribute_ruler", "lemmatizer", "ner")
print("Traitements disponibles : " + ",".join(nlp.pipe_names))


Traitements disponibles : tok2vec,morphologizer,parser,attribute_ruler,lemmatizer,ner
Traitements disponibles : tok2vec,morphologizer


In [3]:
from spacy.lang.fr.examples import sentences

**1b.** L'objet `sentences` chargé ci-dessus contient une liste de phrases en français. 

* Veuillez afficher les deux premières phrases de `sentences`.
* Veuillez analyser chacune de ces deux phrases avec la pipeline `nlp` puis afficher chaque token et son POS tag.
    * indication : aidez-vous de la [documentation](https://spacy.io/models/fr#fr_core_news_sm) de `fr_core_news_sm`
    * consigne d'affichage : indiquer le tag entre crochets après chaque token, comme ceci : Les \[DET\] robots \[NOUN\] ...
    * note : la documentation détaillée du POS tagging dans spaCy est [disponible ici](https://spacy.io/usage/linguistic-features)
* Veuillez commenter la tokenisation et les POS tags observés : vous semblent-ils corrects pour les deux phrases ?

In [4]:
# Veuillez écrire votre code et votre commentaire ici.
print("Deux premières phrases : \n\t" + sentences[0] + "\n\t" + sentences[1])

def format_token(t):
  print(t.text, " [", t.pos_, "]", " ", sep="", end="")
def print_tags(sentence):
  doc = nlp(sentence)
  for t in doc:
    format_token(t)
      
print("\nPOS Tagging : \n\t", end="")
print_tags(sentences[0])
print("\n\t", end="")
print_tags(sentences[1])




Deux premières phrases : 
	Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars
	Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs

POS Tagging : 
	Apple [NOUN] cherche [NOUN] à [ADP] acheter [VERB] une [DET] start [NOUN] - [ADJ] up [ADJ] anglaise [NOUN] pour [ADP] 1 [NUM] milliard [NOUN] de [ADP] dollars [NOUN] 
	Les [DET] voitures [NOUN] autonomes [ADJ] déplacent [ADV] la [DET] responsabilité [NOUN] de [ADP] l' [DET] assurance [NOUN] vers [ADP] les [DET] constructeurs [NOUN] 

Analyse : 

Pour la phrase une, le POS  de "cherche" est NOUN. Il tag également "-" en tant qu'adjectif, ce qui n'a pas de sens. Il devrait annoter "start-up" en tant que nom, en entier.
La deuxième phrase se voit mal annoter le mot "déplacent" en adverbe.

## 2. Prise en main des données

Les données sont fournies dans un format tabulaire dans l'archive `UD_French-GSD.zip` sur Cyberlearn.  Elles sont basées sur les données fournies par le projet [Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD).  Leur format, appelé CoNLL-U, est [documenté ici](https://universaldependencies.org/format.html).  Veuillez placer les trois fichiers contenus dans l'archive dans un sous-dossier de ce notebook nommé `spacy_data`.

Les trois fichiers contiennent des phrases en français annotées avec les POS tags :
* le fichier `fr-ud-train.conllu` est destiné à l'entraînement
* le fichier `fr-ud-dev.conllu` est destiné aux tests préliminaires et aux réglages des paramètres
* le fichier `fr-ud-test.conllu` est destiné à l'évaluation finale.

**2a.** En inspectant les fichiers avec un éditeur texte, veuillez déterminer dans quelle colonne se trouvent les *tokens* des textes originaux, et dans quelle colonne se trouvent leurs étiquettes morpho-syntaxiques correctes (*POS tags*).  Que contient la troisième colonne ?

In [60]:
# col 2 : Tokens des texts originaux
# col 3 : Lemme du mot 
# col 4 : étiquette morpho-syntaxique


**2b.** Veuillez convertir les trois fichiers de données en des fichiers binaires utilisables par spaCy, en utilisant la [commande 'convert' fournie par spaCy](https://spacy.io/api/cli#convert).  La commande est donnée ci-dessous, le premier dossier `./input_data` contient les 3 fichiers `.conllu` et le dossier `./spacy-data` contiendra les 3 résultats.

* Veuillez exécuter la commande de conversion.
* Combien de phrases environ (à 10 phrases près) contient chaque fichier (*train*, *dev*, *test*) ?  Observez la commande et son résultat pour répondre.

In [5]:
!python -m spacy convert ./input_data ./spacy_data --converter conllu  --n-sents 10 --lang fr

[i] Grouping every 10 sentences into a document.
[+] Generated output file (148 documents):
spacy_data\fr-ud-dev.spacy
[i] Grouping every 10 sentences into a document.
[+] Generated output file (42 documents):
spacy_data\fr-ud-test.spacy
[i] Grouping every 10 sentences into a document.
[+] Generated output file (1456 documents):
spacy_data\fr-ud-train.spacy


In [62]:
# Le fichier "dév" contient environ 1480 phrases, le test environ 420 et  le train 14560


**2c**. Les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Dans notre cas, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la première phrase des données de test dans une variable nommée `premiere_phrase_test`.
* Veuillez afficher cette phrase, ainsi que son type dans spaCy.

In [ ]:
from spacy.tokens import DocBin
from spacy.tokens import Doc
test_data = DocBin().from_disk("./spacy_data/fr-ud-test.spacy")
# Exemple d'utilisation (afficher toutes les phrases)
for doc in test_data.get_docs(nlp.vocab): 
   for sent in doc.sents:
       print(sent)

In [7]:
# Veuillez écrire votre code ici.
docs = list(test_data.get_docs(nlp.vocab))
premiere_phrase_test = list(docs[0].sents)[0]
print("Phrase : ", premiere_phrase_test)
print("Type : ", type(premiere_phrase_test))

Phrase :  Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.
Type :  <class 'spacy.tokens.span.Span'>


## 3. Évaluation du POS tagger français de la pipeline `fr_core_news_sm`

**3a.** Veuillez effectuer le *POS tagging* avec spaCy de la `premiere_phrase_test` et afficher les résultats dans le format demandé au (1b).  Indication : convertissez la `premiere_phrase_test` dans un objet de type `Doc` en lui appliquant la méthode `.as_doc()`.  Cet objet peut être ensuite traité par la pipeline `nlp`.

In [8]:
# Veuillez écrire votre code ici.
for token in nlp(premiere_phrase_test.as_doc()):
   format_token(token)

Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [ADJ] fous [ADJ] que [SCONJ] nous [PRON] avons [AUX] déjà [ADV] vus [ADJ] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT] 

**3b.** Veuillez afficher les tags corrects de `premiere_phrase_test`, puis comparez-les visuellement les tags trouvés automatiquement au (3a).  Quelles différences trouvez-vous ?

In [9]:
# Veuillez écrire votre réponse ici.
for token in premiere_phrase_test:
  format_token(token)
  
# On remarque que "scientifiques" est, ici, correctement annoté en tant que "nom" et non pas en tant qu'adjectif comme dans le (3a).
# Egalement, "que" est correctement annoté  en tant que pronom (relatif) et non pas en tant que conjonction.
# "vus" est taggé comme un adverbe alors que c'est un verbe.

Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [NOUN] fous [ADJ] que [PRON] nous [PRON] avons [AUX] déjà [ADV] vus [VERB] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT] 

In [10]:
from spacy.scorer import Scorer
from spacy.training import Example

In [11]:
scorer = Scorer()

**3c.** Au lieu de compter manuellement combien de tags sont différents entre la référence et le résultat de la pipeline `nlp`, vous allez utiliser la classe `Scorer` de spaCy.  Une instance de cette classe permet de calculer les scores d'une liste d'objets de type `Exemple`, en fonction des annotations disponibles dans les objets.  Un objet de type `Exemple` contient deux objets de type `Doc`, l'un avec les annotations correctes et l'autre avec les annotations produites par une pipeline.  La [documentation de la méthode](https://spacy.io/api/scorer#score) `Scorer.score(..)` vous sera utile. 

* Veuillez calculer la justesse (*accuracy*) du *POS tagging* de `premiere_phrase_test`. 
* Veuillez justifier la valeur du score obtenu en utilisant votre réponse du (3b).

In [12]:
# Veuillez écrire votre code ici.
ppt_example = Example(nlp(premiere_phrase_test.__str__()), premiere_phrase_test.as_doc())
score = scorer.score(examples=[ppt_example])
print("Justesse de l'annotation syntaxique : ", score["pos_acc"])
# On a environ 90% d'annotation correcte sur le pipeline custom. 
# Cela correspond au 3 tokens mal annotés sur les 29 tokens de la phrase : 3/29 ~ 10% 

Justesse de l'annotation syntaxique :  0.896551724137931


**3d.** Veuillez calculer la précision du *POS tagging* de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se compare le score obtenu avec celui mentionné [dans la documentation](https://spacy.io/models/fr#fr_core_news_sm) du modèle `fr_core_news_sm` ?

In [13]:
# Veuillez écrire votre code ici, suivi de votre réponse à la question.

examples = [Example(nlp(doc.__str__()), doc) for doc in docs]
total_score = scorer.score(examples=examples)
print("Justesse de l'annotation syntaxique sur toutes les données : ", total_score["pos_acc"])



Justesse de l'annotation syntaxique sur toutes les données :  0.8783971230413562


In [71]:
# Le score obtenu est très légèrement inférieur à celui mentionné dans le modèle, qui est de 0.96. Il semble être sensiblement proche.

## 4. Entraîner puis évaluer un nouveau POS tagger français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent.  Les [instructions d'entraînement](https://spacy.io/usage/training#quickstart) de spaCy vous montrent comment entraîner une pipeline avec un POS tagger.

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous voulez seulement un POS tagger dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [14]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Enfin, veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques, pour estimer le temps nécessaire et observer les messages affichés.  Puis augmentez progressivement le nombre d'époques.  Quel est le critère qui vous permet de décider que vous avez un nombre suffisant d'époques ?  Dans quel dossier se trouve le meilleur modèle ?

In [ ]:
# Pour plus de lisibilité, la commande a été effectuée dans une console séparée. Les différents résultats sont affichés ci-dessous.
!python -m spacy train config.cfg --output ./myPOStagger1 --paths.train ./spacy_data/fr-ud-train.spacy --paths.dev ./spacy_data/fr-ud-dev.spacy --verbose

In [17]:
# Output si max_epoch = 1:
# E    #       LOSS TOK2VEC  LOSS TAGGER  TAG_ACC  SCORE
# ---  ------  ------------  -----------  -------  ------
#   0       0          0.00       211.77    36.34    0.36
#   0     200        315.80     10402.57    90.32    0.90
#   0     400        288.32      4498.76    91.69    0.92
#   0     600        223.74      3454.07    92.22    0.92
#   0     800        216.47      3454.31    92.60    0.93
#   0    1000        190.82      3010.63    92.65    0.93
#   0    1200        185.41      2920.07    92.87    0.93
#   0    1400        171.64      2732.70    93.05    0.93

In [18]:
# Output si max_epoch = 5:
# E    #       LOSS TOK2VEC  LOSS TAGGER  TAG_ACC  SCORE
# ---  ------  ------------  -----------  -------  ------
#   0       0          0.00       211.77    36.34    0.36
#   0     200        315.80     10402.57    90.32    0.90
#   0     400        288.32      4498.76    91.69    0.92
#   0     600        223.74      3454.07    92.22    0.92
#   0     800        216.47      3454.31    92.60    0.93
#   0    1000        190.82      3010.63    92.65    0.93
#   0    1200        185.41      2920.07    92.87    0.93
#   0    1400        171.64      2732.70    93.05    0.93
#   1    1600        146.34      2202.03    93.16    0.93
#   1    1800        137.66      2014.74    93.15    0.93
#   1    2000        155.27      2302.77    93.24    0.93
#   1    2200        146.14      2144.11    93.16    0.93
#   1    2400        148.49      2155.38    93.35    0.93
#   1    2600        149.73      2150.74    93.21    0.93
#   1    2800        150.57      2193.10    93.49    0.93
#   2    3000        132.13      1822.63    93.37    0.93
#   2    3200        120.91      1551.85    93.22    0.93
#   2    3400        135.51      1746.97    93.48    0.93
#   2    3600        129.20      1625.42    93.47    0.93
#   2    3800        126.80      1588.73    93.40    0.93
#   2    4000        132.37      1675.89    93.48    0.93
#   2    4200        141.76      1760.37    93.53    0.94
#   3    4400        139.69      1711.32    93.60    0.94
#   3    4600        102.09      1169.59    93.54    0.94
#   3    4800        114.78      1289.31    93.53    0.94
#   3    5000        128.95      1449.13    93.56    0.94
#   3    5200        124.82      1396.30    93.55    0.94
#   3    5400        124.12      1407.72    93.52    0.94
#   3    5600        124.74      1387.35    93.70    0.94
#   3    5800        131.50      1449.70    93.64    0.94
#   4    6000        111.58      1173.01    93.63    0.94
#   4    6200        103.32      1040.85    93.57    0.94
#   4    6400        108.51      1089.15    93.50    0.93
#   4    6600        117.26      1170.86    93.66    0.94
#   4    6800        121.55      1221.80    93.70    0.94
#   4    7000        122.86      1216.95    93.63    0.94
#   4    7200        115.45      1132.62    93.67    0.94


In [ ]:
# Veuillez indiquer ici le nombre d'époques final et la réponse à la question.
# Le critère qui permet de décider que l'on a un nombre suffisant d'époques est le score de précision qui ne varie plus ou qui diminue.
# Le meilleur modèle se trouve dans le dossier /myPOStagger1/model-best
# Le nombre d'époques final est de 5.


**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher la *POS tagging accuracy* sur le corpus de test.  Le composant de la pipeline étant un *POS tagger*, vous devrez évaluer la propriété *tag_acc*. 

In [20]:
# Veuillez écrire votre code ici.
nlp2 = spacy.load("./myPOStagger1/model-best")
docs = list(test_data.get_docs(nlp2.vocab))
examples = [Example(nlp2(doc.__str__()), doc) for doc in docs]
total_score = scorer.score(examples=examples)
print("Justesse de l'annotation syntaxique sur toutes les données : ", total_score["tag_acc"])


Justesse de l'annotation syntaxique sur toutes les données :  0.9357939649411402


## 5. Entraîner puis évaluer un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger appelé *Averaged Perceptron* fourni par NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus, importées cette fois-ci avec NLTK.  Pour une introduction au POS tagging avec NLTK, voir le [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html).

Remarques :
* pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK ;
* pour appliquer un tagger existant, on écrit `nltk.pos_tag(sentence)` où `sentence` est une liste de tokens et on obtient des paires (token, TAG) ;
* l'implémentation de *Averaged Perceptron* a été faite par [Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python), la société qui a créé spaCy.

**5a.** Veuillez charger les données d'entraînement et celles de test grâce à la classe `ConllCorpusReader` de NLTK.  [La documentation de cette classe](https://www.nltk.org/api/nltk.corpus.reader.conll.html#nltk.corpus.reader.conll.ConllCorpusReader) vous montrera comment indiquer les colonnes qui contiennent les tokens ('words') et les tags corrects ('pos').  Une fois les données chargées dans une variable, vous pouvez accéder aux phrases et aux tags avec la méthode `.tagged_sents()`.

In [21]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [71]:
# Veuillez écrire votre code ici.
train = ConllCorpusReader("./input_data", "fr-ud-train.conllu", ["ignore", "words", "ignore", "pos", "ignore", "ignore", "ignore", "ignore", "ignore", "ignore"], separator="\t")
test = ConllCorpusReader("./input_data", "fr-ud-test.conllu", ["ignore", "words", "ignore", "pos", "ignore", "ignore", "ignore", "ignore", "ignore", "ignore"], separator="\t")
train_sents = train.tagged_sents()
test_sents = test.tagged_sents()

print(train_sents)

[[('Les', 'DET'), ('commotions', 'NOUN'), ('cérébrales', 'ADJ'), ('sont', 'AUX'), ('devenu', 'VERB'), ('si', 'ADV'), ('courantes', 'ADJ'), ('dans', 'ADP'), ('ce', 'DET'), ('sport', 'NOUN'), ("qu'", 'SCONJ'), ('on', 'PRON'), ('les', 'PRON'), ('considére', 'VERB'), ('presque', 'ADV'), ('comme', 'ADP'), ('la', 'DET'), ('routine', 'NOUN'), ('.', 'PUNCT')], [("L'", 'DET'), ('œuvre', 'NOUN'), ('est', 'AUX'), ('située', 'VERB'), ('dans', 'ADP'), ('la', 'DET'), ('galerie', 'NOUN'), ('des', '_'), ('de', 'ADP'), ('les', 'DET'), ('batailles', 'NOUN'), (',', 'PUNCT'), ('dans', 'ADP'), ('le', 'DET'), ('château', 'NOUN'), ('de', 'ADP'), ('Versailles', 'PROPN'), ('.', 'PUNCT')], ...]


**5b.** Pour entraîner un POS tagger du type Averaged Perceptron, vous utiliserez le sous-module `nltk.tag.perceptron` du [module NLTK contenant les taggers](http://www.nltk.org/api/nltk.tag.html).  Les fonctions d'entraînement et de test sont documentées dans ce module.  Après l'entraînement, le réseau de neurones est enregistré dans un fichier `.pickle`, qui est écrasé à chaque entraînement si vous n'en faites pas une copie.  On peut également lire un fichier `.pickle` dans un tagger.

Veuillez écrire le code pour entraîner le POS tagger sur les données d'entraînement.  Comme au (4), pensez augmenter graduellement le nombre d'époques (appelées 'itérations' dans NLTK).

Combien de temps prend l'entraînement ?  Quelle est la taille du fichier enregistré ?

In [72]:
import os
import nltk
# nltk.download('averaged_perceptron_tagger') # à exécuter la première fois
from nltk.tag.perceptron import PerceptronTagger

In [73]:
ptagger = PerceptronTagger(load=False)

In [86]:
# Veuillez écrire votre code ici.
ptagger.train(train_sents, "./myPOStagger2/nr_iter_10.pickle", nr_iter=10)

In [ ]:
# Veuillez écrire ici vos réponses aux questions (temps d'entraînement et taille du modèle).
# Temps à 1 epoch : 12,5 secondes / taille 6.5 Mo
# Temps à 5 epochs: 52 secondes / taille 7.6 Mo
# Temps à 10 epochs: 1 minute 33 secondes  / taille 8.4 Mo


**5c.** Veuillez évaluer le tagger sur les données de test et afficher le taux de correction.

In [87]:
# Veuillez écrire votre code ici.
score = ptagger.accuracy(test_sents)
print("Justesse de l'annotation syntaxique : ", score)

Justesse de l'annotation syntaxique :  0.9588269566906196


## 6. Conclusion

Veuillez remplir le tableau suivant avec les scores obtenus et discuter brièvement comment se comparent les trois POS taggers sur ces données de test.

| spaCy (partie 3)    | spaCy (partie 4)   | NLTK (partie 5)     | 
|---------------------|--------------------|---------------------|
| tagger fourni       | tagger entraîné    | tagger entraîné     |
| 0.8783971230413562  | 0.9357939649411402 | 0.9588269566906196  |


Les scores obtenus montrent que le tagger entraîné avec NLTK est le plus performant, suivi par le tagger entraîné avec spaCy. Le tagger fourni par spaCy est le moins performant.
Pour vraiment attester de l'efficacité des taggers, il faudrait les tester sur un plus grand nombre de phrases issuent de différents corpus. Les taggers entraînés ont l'avantage de pouvoir être adaptés à des corpus spécifiques, ce qui peut les rendre plus performants que les taggers fournis par défaut, mais cela peut également les conditionner sur des corpus spécifiques (overfitting).

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.